In [2]:
import pandas as pd
from os import path

In [3]:
df = pd.read_excel (r'sharedData\200824_Testplan.xlsx', sheet_name='Durchgeführte Messungen')
dfDict = df.to_dict("records")

In [4]:
from mongoTools import connectMongoDB
db = connectMongoDB(True)
db["Testplan"].insert_many(dfDict)

In [10]:
from InFusionTools import convertTime, TIMESTAMPFACTOR
import time
import datetime

dfTestplan = pd.DataFrame(list(db["Testplan"].find()))
for f in dfTestplan["Filename"]:
    f = f.replace("-", "_")
    f = f.replace("c", "k")
    if "verm" in f:
        f = f.split(": ")[1]
    if "Re" in f:
        timestamp = convertTime(f)*TIMESTAMPFACTOR

    else:        
        timestamp = time.mktime(datetime.datetime.strptime(f,
                                                "%d.%m.%Y_%H_%M").timetuple())*TIMESTAMPFACTOR
    dfTestplan["StartID"] = timestamp

In [23]:
#db.Testplan.update_many({}, dfTestplan.to_dict("records"))
db["Testplan_new"].insert_many(dfTestplan.to_dict("records"))

In [27]:
pd.DataFrame(list(db["Testplan_new"].find({"Belag": "Asphalt"}))).head()

,_id,IDs,Nr,Name,Filename,Bemerkung,Messungen,max amp,10 V,Störung,...,Motordrehzahl,Gang,Beschleunigung,Querbeschleunigung,Beladung,Witterung,Review Comment,Change Required,Wasserfilmhöhe,StartID
0,622f1b0d3d1b053fd62df282,1.0,76/157,Sweep trocken 30 km/h,Rekorder_2020-09-01_13-57-08,157-162 wurden fälschlicherweise als nass dekl...,1.0,ja,nein,NaN,...,-,-,0 m/s²,-,-,trocken,Verlauf eventuell zuschneiden (Anfang) fahrbah...,ja,0,1.606998e+14
1,622f1b0d3d1b053fd62df283,2.0,76_2/157_2,Sweep trocken 30 km/h,Rekorder_2020-09-01_13-59-38,NaN,1.0,ja,nein,NaN,...,-,-,0 m/s²,-,-,trocken,"Verlauf zuschneiden, Anfangsgeschwnidigkeit zu...",ja,0,1.606998e+14
2,622f1b0d3d1b053fd62df284,3.0,77/158,Sweep trocken 50 km/h,Rekorder_2020-09-01_14-01-21,NaN,1.0,ja,nein,NaN,...,-,-,0 m/s²,-,-,trocken,Geschwindigkeit schwankt sehr stark 40 km/h-54...,NaN,0,1.606998e+14
3,622f1b0d3d1b053fd62df285,4.0,77_2/158_2,Sweep trocken 50 km/h,Rekorder_2020-09-01_14-03-04,NaN,1.0,ja,nein,NaN,...,-,-,0 m/s²,-,-,trocken,Verlauf zuschneiden,NaN,0,1.606998e+14
4,622f1b0d3d1b053fd62df286,5.0,78/159,Stat. Kreisfahrt (links) trocken 2 m/s²,Rekorder_2020-09-01_13-47-23,NaN,1.0,ja,nein,NaN,...,NaN,NaN,0 m/s²,2 m/s²,-,trocken,Geschwindigkeit zu groß,NaN,0,1.606998e+14
